# YOLOv5 Weak-to-Strong Training

## Setup

In [ ]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="PZvJbpNUYuJJDMh8nypC")
project = rf.workspace("gdit").project("aerial-airport")
dataset = project.version(1).download("yolov5")

In [ ]:
epochs = 50
batch_size = 16
img_size = 600
weak_model = "yolov5n"
strong_model = "yolov5m"

## Train strong ceiling model (YOLOv5m)

In [ ]:
!python train.py --img {img_size} --batch {batch_size} --epochs {epochs} --data {dataset.location}/data.yaml --weights {strong_model}.pt --cache

In [ ]:
from google.colab import files
files.download('./runs/train/exp')

## Train weak model (YOLOv5n)

In [ ]:
reduced_dataset_path = '/content/drive/MyDrive/Colab\ Notebooks/weak-to-strong-generalization/Aerial-Airport-1-reduced'

In [ ]:
!python train.py --img {img_size} --batch {batch_size} --epochs {epochs} --data {reduced_dataset_path}/data.yaml --weights {weak_model}.pt --cache

In [ ]:
from google.colab import files
files.download('./runs/train/exp1')

## Generate weak labels

In [ ]:
!python detect.py --weights runs/train/exp1/weights/best.pt --img 600 --conf 0.1 --source {dataset.location}/train/images --save-txt

In [ ]:
#display inference on ALL test images

# import glob
# from IPython.display import Image, display

# for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
#     display(Image(filename=imageName))
#     print("\n")

In [ ]:
%rm -r {dataset.location}/train/labels
%cp -r runs/detect/exp/labels {dataset.location}/train

## Train strong student model (YOLOv5m)

In [ ]:
!python train.py --img {img_size} --batch {batch_size} --epochs {epochs} --data {dataset.location}/data.yaml --weights {strong_model}.pt --cache

In [ ]:
from google.colab import files
files.download('./runs/train/exp2')